# Dataset generator
This notebook allows you to generate a dataset similar to the one used in the paper.

It assumes that Ollama and the desired LLM are installed on your system.

In [ ]:
import requests
import numpy as np
import pandas as pd

In [ ]:
# Configuration
OLLAMA_HOST = "http://127.0.0.1:11434"  # default Ollama API endpoint
MODEL_NAME = "mistral"                  # must match the local Ollama model


In [ ]:
def ollama_generate(prompt: str, max_tokens:int=5, temperature:float=0.2):
    """
    Sends prompt to Ollama local API and returns text tokens.
    """
    payload = {
        "model": MODEL_NAME,
        "prompt": prompt,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "stream": False
    }

    r = requests.post(f"{OLLAMA_HOST}/v1/completions", json=payload)
    r.raise_for_status()
    response = r.json()
    #print(response)

    # Ollama returns a list of completions (usually just one)
    text = response['choices'][0]['text']
    return text

def clean_first_token(text:str) -> str:
    tokens = text.split()
    return tokens[0].lower().strip().replace(",", "").replace(".", "").replace("\"", "").replace(":", "").replace("!", "").replace("?", "")


In [ ]:
# Agent never sees true label of object, only perceptions of it as produced by the LLM.
# Perceptual questions should be observable phenomena, not facts which require experience to learn.
# Perceptual questions cost time; the agent must eat things when hungry.

# LLM used as world simulation and as LTM

# Create scenarios statistically. 
# Fill in the perceptions from the LLM
# Original list of encounters was generated by LLM but had far too many cats.
# Classes (keys) are observable; precise scenarios are not observable but used in world model
scenario_classes = {
    "maybe something to eat": [
        # Food
        "cheese",
        "tomato",
        "carrot",
        "cauliflower",
        "radish",

        # Poisons
        "deadly nightshade",
        "slug pellets",
        "fly agaric mushroom",
    ],
    "bird": [
        # Predatory birds
        "eagle",
        "hawk",
        "falcon",

        # Harmless birds
        "sparrow",
        "pigeon",
    ],  # yes birds are animals too
    "land animal": [
        # Dangerous?
        "cat",
        "dog",
        "fox",
        "snake",
        "farmer",

        # Harmless mammals
        "beetle",
        "horse",
        "mouse",
        "capybara",
    ],
    "plant": [
        # Plants
        "tree",
        "grass",
    ],
}

actions = [
    "Go to it",
    "Eat it",
    "Hide",
    "Run away",
]
scenario_prompt = "You are a mouse in a garden and you see a"

# Attributes which can be perceived
perceptions = [
    "Does it look like a mouse?",
    "Is it bigger than a mouse?",
    "Does it smell tasty?",
    "Does it have a long tail?",
    "Does it have four legs?",
    "Is it red?",
    "Is it green?",
    "Is it noisy?",
    "Is it watching you?",
    "Is it coming towards you?",
    
]
# Used in environment update to determine reward from actions
attributes = [
    "Is it edible?",  # +ve reward if eaten
    "Is it poisonous?",  # -ve reward if eaten
    "Does it eat mice?",  # -ve reward on go-to or eat-it +ve if hide / run-away
    "Is it friendly?",  # +ve reward if approached; -ve if hide
    "Does it chase mice?", # -ve reward if run away
]


In [ ]:
# Dataset options
temperature = 0.7
num_scenarios = 1000
scenarios_filename = "new_scenarios.csv"

In [ ]:

# Generate 1000 scenarios
scenarios = []
scenario_class_list = list(scenario_classes.keys())
num_classes = len(scenario_class_list)

for i in range(num_scenarios):

    # Pick a random class
    n = np.random.randint(0, num_classes)
    k = scenario_class_list[n]

    # Get specific instances for that class
    # Pick a specific encounter
    scenario_list = list(scenario_classes[k])
    m = np.random.randint(0, len(scenario_list))

    # Add to list of scenarios
    scenario_class = k
    scenario_object = scenario_list[m]
    scenario = [scenario_class, scenario_object]
    scenarios.append(scenario)
    print(f"{i}/{num_scenarios}: {scenario}")


In [ ]:

def sample_answers(thing:str, yes_no_question:str):
    prompt = f"Respond with one word, Yes or No. {scenario_prompt} {thing}. {yes_no_question}"""
    print(prompt)
    n = 0
    max_tries = 10
    while(n < max_tries):
        n += 1
        response = ollama_generate(prompt, temperature=temperature)
        word = clean_first_token(response)
        if word == "yes" or word == "no":
            return word
    return "?"


In [ ]:
col_data = {}

def append_sample(k, v):
    if k not in col_data.keys():
        col_data[k] = []
    col_data[k].append(v)

for i in range(num_scenarios):
    print(f"{i}/{num_scenarios} ")
    object_class = scenarios[i][0]
    object_name = scenarios[i][1]
    append_sample("Class", object_class)
    append_sample("Object", object_name)

    for q in perceptions:
        answer = sample_answers(
            thing = object_name, 
            yes_no_question = q, 
        )
        append_sample(q, answer)
        
    for q in attributes:
        answer = sample_answers(
            thing = object_name, 
            yes_no_question = q, 
        )
        append_sample(q, answer)




In [ ]:
df = pd.DataFrame(col_data)
print(df.head(10))
df.to_csv(scenarios_filename)
